In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
df = pd.read_csv(r"C:\Users\KAVISH\Downloads\FidelFolio_df2.csv")

In [5]:
for col in df.columns:
    if col not in ['Company', 'Year']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        

In [6]:
df['target_lag1'] = df.groupby('Company')['Target 2'].shift(1)
df['Target2_lag1'] = df['target_lag1'].fillna(-999)

In [9]:
feature_cols = [col for col in df.columns if col not in ['Company', 'Year', 'Target 2']]

In [10]:
df[feature_cols] = df[feature_cols].fillna(-999)

In [11]:
SEQ_LEN = 3  # Increased sequence length
MASK_VALUE = -999.
BATCH_SIZE = 64  # Adjusted batch size
EPOCHS = 30  # Increased epochs with early stopping
INIT_LR = 0.0005  # Lower initial learning rate

In [13]:
X_train = df[[col for col in feature_cols if col in df.columns]].values

y_train = df['Target 2'].values

In [15]:

train_df = df[df['Year'] < 2023].copy()
test_df = df[df['Year'] > 2022].copy()


#train_df = train_df.dropna(subset=[' Target 1 '])

# Fill NaNs in features (you can also choose to drop or impute)
X_columns = [col for col in train_df.columns if col not in ['Company', 'Year', 'Target 1','Target 2',  'target 3'] ]
train_df[X_columns] = train_df[X_columns].fillna(-999)
test_df[X_columns] = test_df[X_columns].fillna(-999)


X_train = train_df[X_columns]
y_train = train_df['Target 2'].values

X_test = test_df[X_columns]

# Build the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict for 2024
predictions_2024 = model.predict(X_test)

# Add predictions to test_df
test_df['predicted_target2'] = predictions_2024

# Save the predictions
test_df[['Company', 'Year', 'predicted_target2']].to_csv("2024_predictions.csv", index=False)

Epoch 1/50
603/603 [==============================] - 3s 3ms/step - loss: 68096.9844 - mae: 138.2500 - val_loss: 226881.8906 - val_mae: 135.3683
Epoch 2/50
603/603 [==============================] - 2s 3ms/step - loss: 42728.8828 - mae: 99.6695 - val_loss: 222529.7500 - val_mae: 114.9033
Epoch 3/50
603/603 [==============================] - 2s 3ms/step - loss: 41956.1016 - mae: 96.7002 - val_loss: 227367.4219 - val_mae: 114.0913
Epoch 4/50
603/603 [==============================] - 2s 3ms/step - loss: 41310.1289 - mae: 95.0134 - val_loss: 212728.5312 - val_mae: 120.1916
Epoch 5/50
603/603 [==============================] - 2s 3ms/step - loss: 41431.0781 - mae: 94.3011 - val_loss: 219795.4062 - val_mae: 132.8773
Epoch 6/50
603/603 [==============================] - 2s 4ms/step - loss: 41020.7070 - mae: 93.8000 - val_loss: 210254.7188 - val_mae: 110.7255
Epoch 7/50
603/603 [==============================] - 3s 5ms/step - loss: 41872.4258 - mae: 93.0945 - val_loss: 232758.8750 - val_mae: 

In [17]:
test_df.to_csv('test_data_processed2.csv', index=False)